# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: e4e17e9d-58d2-4c67-bd8c-b4d46ff02255
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session e4e17e9d-58d2-4c67-bd8c-b4d46ff02255 to get into ready status...
Session e4e17e9d-58d2-4c67-bd8c-b4d46ff02255 has been created.



In [3]:
import sys
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.utils import getResolvedOptions

# Initialize Glue context
glueContext = GlueContext(SparkContext.getOrCreate())

# Define S3 paths
input_path = "s3://etherscandata/raw_data/invoke-ethereum_transactions.json"
output_path = "s3://etherscandata/new-parquet/"

# Step 1: Read the JSON file from S3
dynamic_frame = glueContext.create_dynamic_frame.from_options(
    connection_type="s3", 
    format="json", 
    connection_options={"paths": [input_path]}
)

# Step 2: Convert to DataFrame for easier manipulation (optional)
dataframe = dynamic_frame.toDF()

# Step 3: Write the DataFrame as Parquet back to S3
dataframe.write.parquet(output_path)

print("Conversion to Parquet complete!")


Conversion to Parquet complete!


In [7]:
# Correct path based on your input
parquet_file_path = "s3://etherscandata/new-parquet/part-00002-55005096-6b24-4d05-9d1a-3c98ddfd466f-c000.snappy.parquet"

# Step 1: Read the parquet file into a DataFrame
df = spark.read.parquet(parquet_file_path)

# Step 2: Show the data (limit the rows by using the 'limit' function)
df.show(10)  # Show the first 10 rows

# Optionally, display the schema to understand the structure of the data
df.printSchema()


+--------------------+--------------------+--------------------+-----------+--------------------+----------------+--------------------+--------+-------+-------+----+------+
|             address|              topics|                data|blockNumber|     transactionHash|transactionIndex|           blockHash|logIndex|removed|jsonrpc|  id|result|
+--------------------+--------------------+--------------------+-----------+--------------------+----------------+--------------------+--------+-------+-------+----+------+
|0x386b76d9ca5f5fb...|[0x2849b43074093a...|0x000000000000000...|  0x13c09a9|0xbc6c9bbd41648d0...|            0x4f|0x1e1710ec6677b13...|    0xdc|  false|   null|null|  null|
|0x17bfafa932d2e23...|[0xb3813568d9991f...|0x000000000000000...|  0x13c09a9|0xbc6c9bbd41648d0...|            0x4f|0x1e1710ec6677b13...|    0xdd|  false|   null|null|  null|
|0x17bfafa932d2e23...|[0xcfcb62499f5737...|0x000000000000000...|  0x13c09a9|0xbc6c9bbd41648d0...|            0x4f|0x1e1710ec6677b13...|

In [9]:
# Step 1: Read the parquet file into a DataFrame
parquet_file_path = "s3://etherscandata/new-parquet/part-00002-55005096-6b24-4d05-9d1a-3c98ddfd466f-c000.snappy.parquet"
df = spark.read.parquet(parquet_file_path)

# Step 2: Check for columns with NULL values
null_columns = []
for column in df.columns:
    null_count = df.filter(df[column].isNull()).count()
    if null_count > 0:
        null_columns.append((column, null_count))

# Step 3: Print the columns with their corresponding NULL counts
if len(null_columns) > 0:
    print("Columns with NULL values and their counts:")
    for col, count in null_columns:
        print(f"Column: {col}, NULL count: {count}")
else:
    print("No columns contain NULL values.")


Columns with NULL values and their counts:
Column: jsonrpc, NULL count: 79665
Column: id, NULL count: 79665
Column: result, NULL count: 79665
